In [1]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.7 MB/s eta 0:00:00


In [2]:
!pip install auto_gptq -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00


In [39]:
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM


model_name = 'fffrrt/ruGPT-3.5-13B-GPTQ'
model_basename = 'gptq_model-4bit-128g'

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoGPTQForCausalLM.from_quantized(model_name,
        model_basename = model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device="cuda:0",
        use_triton=False,
        quantize_config=None)

In [42]:
text = "Продолжи диалог: Собеседник: блин, поиск по танкам ничего не выдал Ты: У того поста тэг танки стоит, не чеши Собеседник: я искал наши танки по Европе и нихрена похожего не выдало Ты:"

encoded_input = tokenizer(text, return_tensors='pt').to('cuda:0')
output = model.generate(
    **encoded_input,
    num_beams=4,
    max_new_tokens=50,
    no_repeat_ngram_size=2,
    #num_return_sequences=3,
    do_sample=True
)

print(tokenizer.decode(output[0], skip_special_tokens=False))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Продолжи диалог: Собеседник: блин, поиск по танкам ничего не выдал Ты: У того поста тэг танки стоит, не чеши Собеседник: я искал наши танки по Европе и нихрена похожего не выдало Ты: Ну так я и говорю, что это не наши<|endoftext|>
Что такое "танкисты"?
Это те кто на танках катается.
Я так и понял, но в чём разница между танкистами и танкостроителями?
А


In [ ]:
use_gradient_checkpointing = True

for name, param in model.named_parameters():
    param.requires_grad = False

if use_gradient_checkpointing:
    if hasattr(model, "enable_input_require_grads"):
        model.enable_input_require_grads()
    else:
        def make_inputs_require_grad(module, input, output):
            output.requires_grad_(True)
        model.get_input_embeddings().register_forward_hook(make_inputs_require_grad)
    # enable gradient checkpointing for memory efficiency
    model.gradient_checkpointing_enable()

In [3]:
import torch
if torch.cuda.get_device_capability() < (7, 5):
  raise ValueError(f"You got a GPU with capability {torch.cuda.get_device_capability()}, need at least (7, 5)")
else: print("OK")

# Note: this code requires a Turing GPU or newer. Good: T4, RTX 20xx/30xx, A100/Axx; Bad: K80, P100, V100
# Colab gives you T4. If you get older GPUs, please wait or switch to a new account (don't use both at the same time)
%pip install --quiet bitsandbytes==0.35.4 transformers==4.24.0 datasets==2.7.0 accelerate==0.14.0

OK
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 98.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.6/451.6 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
auto-gptq 0.4.2 requires accelerate>=0.19.0, but you have accelerate 0.14.0 which is incompatible.
auto-gptq 0.4.2 requires transformers>=4.31.0, but you have transformers 4.24.0 which is incompatible.


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "ai-forever/ruGPT-3.5-13B", load_in_8bit=True, device_map='auto',
    low_cpu_mem_usage=True, torch_dtype=torch.float16, offload_state_dict=True)
# note: these flags slow down the code to save RAM; remove them if you have >32GB RAM
tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruGPT-3.5-13B")

for module in model.modules():
    if isinstance(module, bnb.nn.Linear8bitLt):
        module.state.memory_efficient_backward = True

for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.model.decoder.project_in = lambda x: x.requires_grad_(True)

# cast model outputs to float32 to unfuck the top-k sampler
class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 7>:7                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:463 in          │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   460 │   │   │   )                                                                              │
│   461 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│   462 │   │   │   model_class = _get_model_class(config, cls._model_mapping)                     │
│ ❱ 463 │   │   │   return model_class.from_pretrained(                                            │
│   464 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   465 │   │   │   )                                                                              │
│   466 │   │   raise ValueError(                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2326 in from_pretrained   │
│                                                                                                  │
│   2323 │   │   │   if dtype_orig is not None:                                                    │
│   2324 │   │   │   │   torch.set_default_dtype(dtype_orig)                                       │
│   2325 │   │   │                                                                                 │
│ ❱ 2326 │   │   │   model, missing_keys, unexpected_keys, mismatched_keys, error_msgs = cls._loa  │
│   2327 │   │   │   │   model,                                                                    │
│   2328 │   │   │   │   state_dict,                                                               │
│   2329 │   │   │   │   loaded_state_dict_keys,  # XXX: rename?                                   │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2550 in                   │
│ _load_pretrained_model                                                                           │
│                                                                                                  │
│   2547 │   │   │   │   )                                                                         │
│   2548 │   │   │   │                                                                             │
│   2549 │   │   │   │   if low_cpu_mem_usage:                                                     │
│ ❱ 2550 │   │   │   │   │   new_error_msgs, offload_index, state_dict_index = _load_state_dict_i  │
│   2551 │   │   │   │   │   │   model_to_load,                                                    │
│   2552 │   │   │   │   │   │   state_dict,                                                       │
│   2553 │   │   │   │   │   │   loaded_keys,                                                      │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:618 in                    │
│ _load_state_dict_into_meta_model                                                                 │
│                                                                                                  │
│    615 │   │   elif not load_in_8bit:                                                            │
│    616 │   │   │   set_module_tensor_to_device(model, param